---  
  
Graded Challenge 3  
  
Nama  : Nicku Rendy Perdana  
Batch : HKC-012  
  
Program ini dibuat untuk melakukan identifikasi produk yang sedang viral di Tokopedia sebagai langkah awal untuk menjajaki potensi bisnis sebagai dropshipper produk yang viral dan berpotensi menguntungkan dengan teknik web scrapping.  
  
---

# A. Tahap 1: Pengantar/_Business Understanding_

**Problem Statement**: Sebagai seorang Data Analyst, saya telah menyadari bahwa bergantung pada satu jenis pendapatan saja tidak cukup. Saya masih belum bisa membuat produk sendiri di tengah keterbatasan modal dan hanya sanggup untuk menyiapkan modal untuk memromosikan barang jualan.

## A1. Kerangka Pemahaman Bisnis SMART

**Specific**: Meningkatkan pemasukan bulanan dengan menjalankan bisnis _dropshipping_ di platform _e-commerce_ Tokopedia.

**Measurable**: Meningkatkan pemasukan bulanan paling sedikit Rp5 juta setiap bulannya.

**Achievable**: Sebagai lulusan _bootcamp_ Data Science Hacktiv8, hal yang perlu dilakukan secara runut ialah sebagaimana berikut.  
- Analisis situasi terkini penjualan produk seblak di Tokopedia dengan menggunakan teknik _web scrapping_ terhadap 10 halaman pertama pencarian produk seblak di Tokopedia. Teknik ini memungkinkan saya untuk mengumpulkan data terkait daftar nama-nama produk yang dijual beserta harga, banyaknya produk terjual, nama toko penjual, lokasi toko, beserta nilai pembeli produk tersebut.  
- Membersihkan data hasil _web scrapping_ ke dalam bentuk tabel yang dapat dieksplore, diolah, dan dianalisis  
- Melakukan analisis statistik deskriptif untuk menghasilkan kesimpulan sebaran dan kecenderungan data-data harga, banyak produk terjual, dan penilaian pembeli produk seblak. 
- Melakukan analisis statistik inferensial untuk menghasilkan kesimpulan produk mana yang banyak terjual, produk yang berpotensi laris, dan potensi keuntungan yang didapat.

**Relevant**:  
- Seblak menjadi salah satu jajanan/produk yang kini sedang viral dicari dan diperbincangkan di berbagai ranah daring maupun luring.
- Belanja daring menjadi solusi untuk memperoleh barang yang hanya tersedia/lebih murah untuk diperoleh dari area-area tertentu.

**Time-bound**: Tujuan ini akan ditetapkan untuk berlangsung selama 3 bulan pertama sejak memulai bisnis _dropshipping_ guna memberikan waktu untuk mencapai target pemasukan bulanan tambahan sebesar Rp 5 juta.

# B. Tahap 2: _Web Scraping_

## B1. Kunjungi dan _scrap_ situs web  
  
- Diketahui, alamat web ketika melakukan pencarian kata kunci `seblak` adalah: https://www.tokopedia.com/search?navsource=&page=1&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=.
- Ketika berpindah ke halaman selanjutnya (contoh: halaman 2), alamat web berubah menjadi https://www.tokopedia.com/search?navsource=&page=2&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=.
- Alamat website dapat di-loop secara otomatis dengan koding python karena berpola pada string 'page={_}'  
- Lakukan _scrapping_ terhadap teks yang ada di dalam elemen-elemen html untuk info `Nama Produk`, `Harga Produk`, `Penjual`, `Kota Toko`, `Banyaknya Terjual`, dan `Rating Produk`.


In [3]:
# Load modul-modul yang dibutuhkan

from selenium import webdriver # impor paket `selenium` untuk menjalankan `webdriver`
from bs4 import BeautifulSoup # impor paket `bs4` untuk menjalankan `BeautifulSoup`
# import requests 
import time # impor modul `time` untuk menjalankan proses berkaitan dengan penyimpanan nilai waktu
import pandas as pd # impor modul `pandas` untuk alat analisis tabel/dataframe
from datetime import datetime # impor paket `datetime` untuk menjalankan proses yang berkaitan dengan waktu dan tanggal
import pytz # impor modul `pytz` untuk menjalankan proses yang berkaitan dengan info soal zona waktu
from scipy import stats # impor paket `pytz` untuk menjalankan `stats`
import numpy as np # import modul `numpy`

In [10]:
jakartaTz = pytz.timezone('Asia/Jakarta') # simpan info zona waktu 'Asia/jakarta' ke variabel `jakartaTz`

driver = webdriver.Chrome() # jalankan webdriver Chrome

listNames = [] # List kosong untuk menampung daftar nama produk hasil scrapping
listPrices = [] # List kosong untuk menampung daftar harga produk hasil scrapping
listSellers = [] # List kosong untuk menampung daftar nama toko penjual produk hasil scrapping
listLocations = [] # List kosong untuk menampung daftar lokasi toko penjual produk hasil scrapping
listSolds = [] # List kosong untuk menampung daftar total produk yang sudah terjual dari masing-masing produk hasil scrapping
listRates = [] # List kosong untuk menampung daftar penilaian pembeli hasil scrapping

pagesToScrap = 10 # simpan info berapa banyak halaman yang akan di-scrap ke variabel `pageToScrap`

# jalankan loop untuk setiap range dari halaman pertama hingga ke-sekian yang tersimpan di dalam variabel `pageToScrap`
for pageNumber in range(1,pagesToScrap + 1):
    timeInJakarta = datetime.now(jakartaTz) # simpan informasi waktu terkini untuk zona waktu Asia/Jakarta
    
    currentTime = timeInJakarta.strftime("%Y/%m/%d, %H:%M:%S") # simpan informasi waktu terkini dalam format YYYY/mm/dd, HH:MM:SS
    
    print(f'Start page scrapping at targeted url page {pageNumber} on {currentTime}') # Keluarkan output keterangan waktu saat memulai scrapping halaman terkait
    url = f'https://www.tokopedia.com/search?navsource=&page={pageNumber}&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=' # simpan alamat web untuk halaman yang di-loop dengan variabel `pageNumber`

    driver.get(url) # buka url dengan webbrowser
    
    # masuk ke algoritma untuk melakukan automasi scroll halaman dengan webdriver 
    screenHeight = driver.execute_script('return window.screen.height;') # simpan nilai tinggi jendela browser sebagai basis untuk melakukan scroll dengan jumlah interval yang disimpan pada variabel `interval` 
    interval = 1 # jumlah interval untuk melakukan beberapa kali scroll
    
    # jalankan loop ketika kondisi di dalam ini `True` untuk melakukan tindakan scroll halaman di webdriver
    while True:
        driver.execute_script(f'window.scrollTo(0,{screenHeight * interval});') # jalankan script untuk melakukan scroll ke posisi berdasarkan info tinggi jendela browser dikalikan dengan interval saat ini 
        interval += 1 # tambahkan angka satu secara langsung ke variabel interval
        time.sleep(2) # lakukan jeda dua detik sebelum melanjutkan ke line script berikutnya
        scrollHeight = driver.execute_script('return document.body.scrollHeight;') # simpan info nilai posisi scroll saat ini di halaman web
        
        # lakukan validasi bila nilai dari tinggi jendela webbrowser yang dikalikan dengan interval ke-n saat ini sudah lebih besar nilainya dibanding nilai panjang halaman web yang sedang dibuka, stop loop. Bila tidak memenuhi, lanjutkan loop
        if (screenHeight * interval) > scrollHeight: 
            break
        else:
            pass       
        
    # setelah selesai memroses halaman secara menyeluruh dengan scroll, simpan page source halaman web
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # baca secara spesifik elemen yang memuat grid berisi panel-panel barang yang dijual selain barang-barang promosi
    container = soup.find('div', {'data-testid':'divSRPContentProducts',
                                'data-ssr':'contentProductsSRPSSR'})
    
    # baca seluruh subkotak yang berisi pengelompokkan beberapa produk yang termuat dalam `container`, simpan ke dalam list
    cards = container.find_all('div', {'class':'css-jza1fo'})
    
    lenCards = list(range(len(cards))) # simpan berapa banyak kelompok/subkotak dalam `container`
    
    # jalankan loop untuk membaca produk secara satu persatu pada masing-masing subkotak berdasarkan range banyaknya subkotak
    for indexC in lenCards:
        product = cards[indexC].find_all('div', {'class':'css-llwpbs'}) # simpan ke dalam list elemen untuk produk-produk satu persatu
        
        lenProduct = list(range(len(product))) # simpan banyaknya item dalam list `product`
        
        # jalankan loop untuk membaca beberapa elemen pada masing-masing produk
        for indexP in lenProduct:
            
            # simpan isi nama produk dari teks dalam elemen <div class="prd_link-product-name css-3um8ox" data-testid="spnSRPProdName">
            try:
                name = product[indexP].find('div', {'class':"prd_link-product-name css-3um8ox",
                                    'data-testid':"spnSRPProdName"}).get_text()
                listNames.append(name)
            except: # jalankan script bila tidak ada isi teks
                listNames.append(None)
            
            # simpan isi harga produk dari teks dalam elemen <div class="prd_link-product-price css-h66vau" data-testid="spnSRPProdPrice">
            try:
                price = product[indexP].find('div', {'class':'prd_link-product-price css-h66vau',
                                                    'data-testid':'spnSRPProdPrice'}).get_text()
                listPrices.append(price)
            except: # jalankan script bila tidak ada isi teks
                listPrices.append(None)
                
            # simpan isi nama penjual dari teks dalam elemen <span class='prd_link-shop-name css-1kdc32b flip' data-testid=''>
            try:
                seller = product[indexP].find('span', {'class':'prd_link-shop-name css-1kdc32b flip',
                                                    'data-testid':''}).get_text()
                listSellers.append(seller)
            except: # jalankan script bila tidak ada isi teks
                listSellers.append(None)
            
            # simpan isi kota toko dari teks dalam elemen <span class="prd_link-shop-loc css-1kdc32b flip" data-testid="spnSRPProdTabShopLoc">
            try:
                location = product[indexP].find('span', {'class':'prd_link-shop-loc css-1kdc32b flip',
                                                        'data-testid':'spnSRPProdTabShopLoc'}).get_text()
                listLocations.append(location)
            except: # jalankan script bila tidak ada isi teks
                listLocations.append(None)
                
            # simpan isi banyaknya terjual dari teks dalam elemen <span class="prd_label-integrity css-1sgek4h" data-testid="">
            try:
                sold = product[indexP].find('span', {'class':'prd_label-integrity css-1sgek4h',
                                                        'data-testid':''}).get_text()
                listSolds.append(sold)
            except: # jalankan script bila tidak ada isi teks
                listSolds.append(None)
                
            # simpan isi rating produk dari teks dalam elemen <span class="prd_rating-average-text css-t70v7i" data-testid="">
            try:
                rate = product[indexP].find('span', {'class':'prd_rating-average-text css-t70v7i',
                                                        'data-testid':''}).get_text()
                listRates.append(rate)
            except: # jalankan script bila tidak ada isi teks
                listRates.append(None)
    
    timeInJakarta = datetime.now(jakartaTz) # simpan informasi waktu terkini untuk zona waktu Asia/Jakarta
    currentTime = timeInJakarta.strftime("%Y/%m/%d, %H:%M:%S") # simpan informasi waktu terkini dalam format YYYY/mm/dd, HH:MM:SS
    print(f'>> Completed page scrapping at targeted url page {pageNumber} on {currentTime}') # tampilkan pesan sukses melakukan webscraping untuk halaman terkait disertai dengan waktu final eksekusinya

print('Web scrapping completed') # tampilkan output pesan sukses dalam melakukan web scrapping

Start page scrapping at targeted url page 1 on 2024/01/05, 14:00:58
>> Completed page scrapping at targeted url page 1 on 2024/01/05, 14:01:19
Start page scrapping at targeted url page 2 on 2024/01/05, 14:01:19
>> Completed page scrapping at targeted url page 2 on 2024/01/05, 14:01:39
Start page scrapping at targeted url page 3 on 2024/01/05, 14:01:39
>> Completed page scrapping at targeted url page 3 on 2024/01/05, 14:01:57
Start page scrapping at targeted url page 4 on 2024/01/05, 14:01:57
>> Completed page scrapping at targeted url page 4 on 2024/01/05, 14:02:14
Start page scrapping at targeted url page 5 on 2024/01/05, 14:02:14
>> Completed page scrapping at targeted url page 5 on 2024/01/05, 14:02:31
Start page scrapping at targeted url page 6 on 2024/01/05, 14:02:31
>> Completed page scrapping at targeted url page 6 on 2024/01/05, 14:02:52
Start page scrapping at targeted url page 7 on 2024/01/05, 14:02:52
>> Completed page scrapping at targeted url page 7 on 2024/01/05, 14:03:09

## B2. Simpan hasil scrape  
  
Setelah melakukan _web scrapping_, serangkaian list-list yang telah tersimpan perlu disusun menjadi sebuah tabel dengan judul kolom yang berisi data terkait `nama_produk`, `harga`, `toko`, `lokasi_toko`, `terjual`, dan `nilai_produk`. Simpan hasil tabel ke dalam variabel `products_df`, lalu simpan tabel tersebut ke dalam file CSV bernama `products.csv`

In [47]:
data = {'nama_produk':listNames, # buat kolom `nama_produk` yang diisi dengan data dari `listNames`
        'harga':listPrices, # buat kolom `harga` yang diisi dengan data dari `listPrices`
        'toko':listSellers, # buat kolom `toko` yang diisi dengan data dari `listSellers`
        'lokasi_toko':listLocations, # buat kolom `lokasi_toko` yang diisi dengan data dari `listLocations`
        'terjual':listSolds, # buat kolom `terjual` yang diisi dengan data dari `listSolds`
        'nilai_produk':listRates} # buat kolom `nilai_produk` yang diisi dengan data dari `listRates`

products_df = pd.DataFrame(data=data)

In [48]:
# simpan hasil scraping ke dalam file CSV 

products_df.to_csv('products.csv', index=False)

# C. Tahap 2: Persiapan Data

## C1. Cek isi beberapa record data dalam tabel

In [49]:
products_df.head(10)

,nama_produk,harga,toko,lokasi_toko,terjual,nilai_produk
0,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,Rp13.000,Lidigeli,Kab. Garut,250+ terjual,4.8
1,Kylafood Seblak Original Play,Rp15.050,kylafood,Bandung,1rb+ terjual,5.0
2,"Seblak Rafael, Seblak Coet Instan Halal",Rp25.000,Brother Meat Shop,Depok,100+ terjual,4.9
3,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,Rp17.000,Central Seblak Nusantara,Tangerang Selatan,2rb+ terjual,4.9
4,KERUPUK SEBLAK MENTAH ANEKA WARNA,Rp11.500,jajanangarut19,Jakarta Selatan,250+ terjual,4.9
5,SEBLAK INSTANT TERMURAH,Rp3.999,Pusat Cuankie & Cemilan Frozen,Jakarta Timur,10rb+ terjual,4.8
6,"Seblak Rafael, Seblak Coet Instan Halal",Rp25.000,Brother Meat Shop,Depok,100+ terjual,4.9
7,Seblak Instan Pedas Home Made,Rp3.500,the Dhecip,Tangerang Selatan,2rb+ terjual,4.8
8,Seblak pedas kerupuk pedas kiloan 1 kg,Rp35.000,RIKI STORE 97,Kab. Bogor,100+ terjual,4.8
9,seblak instant sajodo snak,Rp19.000,hikmah berkah jaya,Jakarta Barat,750+ terjual,4.8


Deskripsi: Output kode (tabel) tersebut memuat 10 record data seputar daftar nama-nama produk seblak yang dijual pada 10 halaman pertama pencarian di Tokopedia. Beberapa hal yang perlu dilakukan berdasarkan pengamatan tabel saat ini ialah:  
- Pembersihan kolom `harga` dari string "Rp" dan "."
- Pembersihan kolom `terjual` dari string "rb", "+", dan "terjual"
- Pada kolom `terjual`, string "rb" perlu diubah menjadi "000" agar isian ribuan bisa dikonversi langsung ke dalam bentuk integer
- Nilai null/None perlu dibersihkan

## C2. Cek banyaknya isian pada semua kolom

In [54]:
# Baca tabel melalui csv

products_df = pd.read_csv('products.csv')

In [55]:
columns = list(products_df.columns) # buat daftar judul kolom

# jalankan loop untuk print jumlah isi data masing-masing kolom yang dilist secara iteratif
for column in columns:
    print(f"Jumlah data pada kolom '{column}': {len(products_df[column])} data.")

Jumlah data pada kolom 'nama_produk': 821 data.
Jumlah data pada kolom 'harga': 821 data.
Jumlah data pada kolom 'toko': 821 data.
Jumlah data pada kolom 'lokasi_toko': 821 data.
Jumlah data pada kolom 'terjual': 821 data.
Jumlah data pada kolom 'nilai_produk': 821 data.


Deskripsi: Record pada tabel `products_df` memiliki entry yang seimbang, yakni 821 data di seluruh kolom.

## C3. Cek tipe data masing-masing kolom

In [56]:
products_df.info() # cek status value data masing-masing kolom beserta tipe datanya

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821 entries, 0 to 820
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nama_produk   821 non-null    object 
 1   harga         821 non-null    object 
 2   toko          821 non-null    object 
 3   lokasi_toko   821 non-null    object 
 4   terjual       790 non-null    object 
 5   nilai_produk  765 non-null    float64
dtypes: float64(1), object(5)
memory usage: 38.6+ KB


Deskripsi:  
- Diketahui tipe data seluruh kolom masih berupa string.
- Data dalam kolom `price` dan `sold` perlu dikonversi ke dalam bentuk integer.
- Data dalam kolom `rating` perlu dikonversi ke dalam bentuk float.

## C4. Cek banyaknya nilai null di setiap kolom

In [57]:
# jalankan loop untuk menampilkan jumlah nilai null masing-masing kolom
for column in columns:
    print(f"Banyaknya nilai null di kolom '{column}': {products_df[column].isnull().sum()} data.")

Banyaknya nilai null di kolom 'nama_produk': 0 data.
Banyaknya nilai null di kolom 'harga': 0 data.
Banyaknya nilai null di kolom 'toko': 0 data.
Banyaknya nilai null di kolom 'lokasi_toko': 0 data.
Banyaknya nilai null di kolom 'terjual': 31 data.
Banyaknya nilai null di kolom 'nilai_produk': 56 data.


Deskripsi: data pada kolom-kolom `nama_produk`, `harga`, `toko`, dan `lokasi_toko` merupakan seluruhnya data yang valid. Sementara itu, ada sebanyak 31 data di dalam kolom `terjual` dan 56 data di dalam kolom `rating`.  
  
Adanya record produk yang memiliki data kolom `terjual` yang null mengindikasikan bahwa barang yang bersangkutan (dalam record tersebut) belum pernah terjual atau merupakan produk baru yang belum dibeli pengguna Tokopedia.  
  
Begitu pula dengan produk-produk yang memiliki nilai null dalam kolom `nilai_produk`, produk tersebut bisa jadi merupakan produk baru yang belum sempat dibeli per dilakukannya _web scrapping_; bisa jadi pernah dibeli tetapi pembeli tidak ada yang memberikan _rating_/ulasan produk.

## C5. Hapus record dengan nilai null

In [75]:
productsDropNull_df = products_df.copy() # salin dataframe untuk menyimpan proses penghapusan record dengan nilai null

productsDropNull_df = productsDropNull_df.dropna(axis=0).reset_index(drop=True) # hapus record data yang memiliki data null

productsDropNull_df.info() # cek isian data (jumlah non-null) dan tipe data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nama_produk   765 non-null    object 
 1   harga         765 non-null    object 
 2   toko          765 non-null    object 
 3   lokasi_toko   765 non-null    object 
 4   terjual       765 non-null    object 
 5   nilai_produk  765 non-null    float64
dtypes: float64(1), object(5)
memory usage: 36.0+ KB


Deskripsi: Setelah dilakukan pembersihan data null di dalam dataframe, jumlah record data menjadi 765 dari yang semula 821. Hal ini menandakan ada sebanyak 56 data null atau sekitar 6,82% dari keseluruhan data yang tidak dapat diproses lebih lanjut.

## C6. Cek dan hapus data duplikat

In [65]:
print(f'Total record data duplikat: {productsDropNull_df.duplicated().sum()} data') # Tampilkan total data yang terduplikasi di dalam dataframe `productsDropNull_df`

Total record data duplikat: 104 data


Deskripsi: Tabel dalam variabel `productsDropNull` masih memiliki sebanyak 104 data duplikat. Data ini perlu dibersihkan sebelum bisa masuk ke tahap pembersihan data selanjutnya.

In [77]:
productsDropDuplicates_df = productsDropNull_df.copy() # salin dataframe untuk menyimpan tabel yang sudah bersih dari duplikasi

productsDropDuplicates_df = productsDropDuplicates_df.drop_duplicates().reset_index(drop=True) # hapus duplikat dengan mereset angka indeks tabel

productsDropDuplicates_df.info() # tampilkan deskripsi tabel

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nama_produk   661 non-null    object 
 1   harga         661 non-null    object 
 2   toko          661 non-null    object 
 3   lokasi_toko   661 non-null    object 
 4   terjual       661 non-null    object 
 5   nilai_produk  661 non-null    float64
dtypes: float64(1), object(5)
memory usage: 31.1+ KB


Deskripsi: Sebanyak 104 data duplikat telah dibersihkan dari tabel `productsDropDuplicates_df`. Jumlah record data/entry pada tabel tersisa 661 dari hasil _scrapping_ di sebanyak 10 halaman.

## C7. Mengubah tipe data yang sesuai untuk masing-masing kolom

In [173]:
productsClean_df = productsDropDuplicates_df.copy() # salin dataframe `productsDropDuplicates_df` untuk menyimpan tabel yang sudah dibersihkan entry dan disesuaikan tipe datanya

productsClean_df['harga'] = productsClean_df['harga'].str.replace('Rp', '').str.replace('.', '').astype('int64') # bersihkan entry data kolom `harga` dari string "Rp" dan "." agar bisa langsung dikonversikan ke tipe data integer

productsClean_df['terjual'] = productsClean_df['terjual'].str.replace('+', '').str.replace('rb', '000').str.replace('.', '').str.replace(' terjual', '').astype('int64') # bersihkan entry data kolom `terjual` dari string "+", ".", dan " terjual"; serta ubah string "rb" menjadi "000" agar bisa langsung dikonversikan ke tipe data integer

productsClean_df['nilai_produk'] = productsClean_df['nilai_produk'].astype('float64')  # ubah langsung tipe data kolom `nilai_produk` menjadi float number

productsClean_df.head(10) # tampilkan 10 data tabel pertama

,nama_produk,harga,toko,lokasi_toko,terjual,nilai_produk
0,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,13000,Lidigeli,Kab. Garut,250,4.8
1,Kylafood Seblak Original Play,15050,kylafood,Bandung,1000,5.0
2,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,100,4.9
3,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,17000,Central Seblak Nusantara,Tangerang Selatan,2000,4.9
4,KERUPUK SEBLAK MENTAH ANEKA WARNA,11500,jajanangarut19,Jakarta Selatan,250,4.9
5,SEBLAK INSTANT TERMURAH,3999,Pusat Cuankie & Cemilan Frozen,Jakarta Timur,10000,4.8
6,Seblak Instan Pedas Home Made,3500,the Dhecip,Tangerang Selatan,2000,4.8
7,Seblak pedas kerupuk pedas kiloan 1 kg,35000,RIKI STORE 97,Kab. Bogor,100,4.8
8,seblak instant sajodo snak,19000,hikmah berkah jaya,Jakarta Barat,750,4.8
9,"Kylafood Seblak Cup Instan, Halal",11500,Brother Meat Shop,Depok,60,5.0


In [90]:
productsClean_df.info() # cek deskripsi (tipe data) tabel

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nama_produk   661 non-null    object 
 1   harga         661 non-null    int64  
 2   toko          661 non-null    object 
 3   lokasi_toko   661 non-null    object 
 4   terjual       661 non-null    int64  
 5   nilai_produk  661 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 31.1+ KB


Deskripsi: Setelah melakukan pembersihan tipe data kolom dari berbagai macam string yang tidak perlu, didapati bahwa kolom-kolom tersebut dapat dikonversi secara langsung menjadi tipe data pasti, antara lain:  
- `nama_produk`: teks (string/object)
- `harga`: angka (integer)
- `toko`: teks (string/object)
- `lokasi_toko`: teks (string/object)
- `terjual`: angka (integer)
- `nilai_produk`: desimal (float)  
  
Sebanyak 1 kolom memiliki tipe data float, 2 kolom bertipe data integer, dan 3 kolom lainnya merupakan kolom bertipe data string/object.

## C8. Tambahkan kolom baru berisi keterangan lokasi toko di dalam/luar Jabodetabek

In [178]:
listArea = [] # variabel penampung iterasi 
columnSource = productsClean_df['lokasi_toko'] # variabel penampung seluruh entry pada kolom `lokasi_toko`

# jalankan loop ini untuk mengidentifikasikan kesesuaian isi string di `columnSource` dengan kondisi
for entry in columnSource:
    if 'Jakarta' in entry:
        listArea.append('Jabodetabek') # masukkan string 'Jabodetabek' pada list `listArea` bila pada entry terdapat string 'Jakarta'
    elif 'Bogor' in entry:
        listArea.append('Jabodetabek') # masukkan string 'Jabodetabek' pada list `listArea` bila pada entry terdapat string 'Bogor'
    elif 'Depok' in entry:
        listArea.append('Jabodetabek') # masukkan string 'Jabodetabek' pada list `listArea` bila pada entry terdapat string 'Depok'
    elif 'Tangerang' in entry:
        listArea.append('Jabodetabek') # masukkan string 'Jabodetabek' pada list `listArea` bila pada entry terdapat string 'Tangerang'
    elif 'Bekasi' in entry:
        listArea.append('Jabodetabek') # masukkan string 'Jabodetabek' pada list `listArea` bila pada entry terdapat string 'Bekasi'
    else:
        listArea.append('Luar Jabodetabek') # masukkan string 'Luar Jabodetabek' pada list `listArea` bila seluruh kondisi sebelumnya tidak terpenuhi
        
productsClean_df['area'] = listArea # buat kolom baru berisi status area lokasi toko

productsClean_df.head(5) # cek hasil penambahan kolom



,nama_produk,harga,toko,lokasi_toko,terjual,nilai_produk,area
0,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,13000,Lidigeli,Kab. Garut,250,4.8,Luar Jabodetabek
1,Kylafood Seblak Original Play,15050,kylafood,Bandung,1000,5.0,Luar Jabodetabek
2,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,100,4.9,Jabodetabek
3,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,17000,Central Seblak Nusantara,Tangerang Selatan,2000,4.9,Jabodetabek
4,KERUPUK SEBLAK MENTAH ANEKA WARNA,11500,jajanangarut19,Jakarta Selatan,250,4.9,Jabodetabek


Deskripsi: Tabel `productsClean_df` yang baru ini telah memiliki indikator nilai yang menunjukkan baik lokasi toko penjual toko terletak di area Jabodetabek maupun di luar Jabodetabek.

In [53]:
lenMetroArea = len(productsClean_df[productsClean_df['area'] == 'Jabodetabek']) # simpan nilai jumlah produk yang pada kolom `area` terisi "Jabodetabek"
lenNonMetroArea = len(productsClean_df[productsClean_df['area'] == 'Luar Jabodetabek']) # simpan nilai jumlah produk yang pada kolom `area` terisi "Luar Jabodetabek"


print(f'Jumlah produk yang dipasarkan di area Jabodetabek: {lenMetroArea} produk.')
print(f'Jumlah produk yang dipasarkan di area luar Jabodetabek: {lenNonMetroArea} produk.')

Jumlah produk yang dipasarkan di area Jabodetabek: 422 produk.
Jumlah produk yang dipasarkan di area luar Jabodetabek: 239 produk.


Deskripsi: Terdapat 422 produk dari 661 produk yang dipasarkan oleh toko-toko yang berasal dari area Jabodetabek. Sementara itu, sisanya, sebanyak 239 produk dipasarkan dari toko-toko di luar jabodetabek.

In [241]:
uniqueShopMetro = len(productsClean_df[productsClean_df['area'] == 'Jabodetabek']['toko'].unique()) # simpan total toko (unique value) yang pada kolom `area` terisi "Jabodetabek"
uniqueShopNonMetro = len(productsClean_df[productsClean_df['area'] == 'Luar Jabodetabek']['toko'].unique()) # simpan total toko (unique value) yang pada kolom `area` terisi "Luar Jabodetabek"

print(f'Jumlah toko dari area Jabodetabek: {uniqueShopMetro} toko.')
print(f'Jumlah toko dari area luar Jabodetabek: {uniqueShopNonMetro} toko.')

Jumlah toko dari area Jabodetabek: 199 toko.
Jumlah toko dari area luar Jabodetabek: 122 toko.


Deskripsi: Terdapat 199 toko dari 321 toko yang menjajakan dagangan seblak di Tokopedia pada 10 halaman pencarian pertama. Toko dari area Jabodetabek mengambil porsi sebanyak 61,66% pasar seblak di Tokopedia.  
  
Sementara itu, terdapat 122 toko yang menjajakan dagangan seblak di Tokopedia pada 10 halaman pencarian pertama. Toko tersebut mengambil pangsa pasar sebesar 38,01% penjual seblak di Tokopedia.  
  
Hal ini menyimpulkan, mayoritas aktivitas distribusi produk seblak dari produsen/distributor ke konsumen masih terpusat di area Jabodetabek.

## C9: Simpan Hasil pembersihan data ke dalam format CSV

In [242]:
# simpan tabel dalam `productsClean_df` ke dalam tabel csv `productsCleanTable.csv`
productsClean_df.to_csv('productsCleanTable.csv', index=False)

# D. Analisis Data

In [4]:
# Load data tabel csv yang sudah bersih

productsClean_df = pd.read_csv('productsCleanTable.csv')

productsClean_df

,nama_produk,harga,toko,lokasi_toko,terjual,nilai_produk,area
0,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,13000,Lidigeli,Kab. Garut,250,4.8,Luar Jabodetabek
1,Kylafood Seblak Original Play,15050,kylafood,Bandung,1000,5.0,Luar Jabodetabek
2,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,100,4.9,Jabodetabek
3,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,17000,Central Seblak Nusantara,Tangerang Selatan,2000,4.9,Jabodetabek
4,KERUPUK SEBLAK MENTAH ANEKA WARNA,11500,jajanangarut19,Jakarta Selatan,250,4.9,Jabodetabek
...,...,...,...,...,...,...,...
656,SEBLAK JUARA INSTAN MASAK BASAH ASLI BANDUNG E...,22000,Rasa Juara Indonesia,Kab. Bandung,1000,4.9,Luar Jabodetabek
657,seblak kering bumbu kacang(100gr),4999,seblak_joy,Kab. Bandung,20,5.0,Luar Jabodetabek
658,Kerupuk Rafael/kerupuk seblak viral/kerupuk ma...,33000,KEDAI SAYUR KU,Tangerang Selatan,30,4.9,Jabodetabek
659,Krupuk Seblak Kencur Seblak Jablay 250gr,12500,Mokosnacksby,Surabaya,250,4.9,Luar Jabodetabek


## D1. Analisis Statistik Deskriptif

In [5]:
priceColumn = productsClean_df['harga']

meanPrice = priceColumn.mean()
medianPrice = priceColumn.median()
modePrice = priceColumn.mode()[0]
minPrice = priceColumn.min()
maxPrice = priceColumn.max()
stdevPrice = priceColumn.std()
skewnessPrice = priceColumn.skew()
kurtPrice = priceColumn.kurt()

print(f'Rata-rata harga seblak di Tokopedia (Rp): {meanPrice}')
print(f'Nilai tengah harga seblak di Tokopedia (Rp): {medianPrice}')
print(f'Modus dari nilai harga seblak di Tokopedia (Rp): {modePrice}')
print(f'Harga seblak terendah di Tokopedia (Rp): {minPrice}')
print(f'Harga seblak tertinggi di Tokopedia (Rp): {maxPrice}')
print(f'Rentang harga seblak di Tokopedia (Rp): {maxPrice - minPrice}')
print(f'Standar deviasi harga seblak di Tokopedia (Rp): {stdevPrice}')
print(f'Arah kecenderungan skewness harga seblak di Tokopedia: {skewnessPrice}')
print(f'Arah kecenderungan kurtosis harga seblak di Tokopedia: {kurtPrice}')

Rata-rata harga seblak di Tokopedia (Rp): 23028.934947049926
Nilai tengah harga seblak di Tokopedia (Rp): 15000.0
Modus dari nilai harga seblak di Tokopedia (Rp): 5000
Harga seblak terendah di Tokopedia (Rp): 350
Harga seblak tertinggi di Tokopedia (Rp): 216630
Rentang harga seblak di Tokopedia (Rp): 216280
Standar deviasi harga seblak di Tokopedia (Rp): 25441.67290103655
Arah kecenderungan skewness harga seblak di Tokopedia: 3.267570070682544
Arah kecenderungan kurtosis harga seblak di Tokopedia: 14.45695513539628


Deskripsi:  
Dari pencarian di sepuluh halaman pencarian 'seblak' di Tokopedia, diketahui rerata harga dari seluruh produk yang dijual ialah sekitar Rp23.028,93. Harga produk seblak paling murah ialah Rp350 dan yang termahal ialah Rp216.630. Nilai tengah dari serangkaian rentang harga tersebut ialah ada di angka Rp15.000,00. Harga seblak yang umumnya dijual ialah di harga Rp5.000 per satuan.  
  
Melihat nilai `median`< `mean` kolom `harga`, diketahui distribusi harga produk seblak dengan sampel data 661 cenderung condong (_skew_) ke kiri, dengan ragam outlier yang cenderung banyak di kanan (nilai positif). Hal ini dibuktikan dengan nilai _skewness_ pada data di kolom `harga` yang bernilai 3,2. Hal ini menunjukkan distribusi sampel data cenderung tidak normal.  

Banyaknya _outlier_ juga terdeteksi nilai kurtosis yang ada di angka 14,45, yang mengindikasikan besarnya jumlah data yang merupakan _outlier_ data.  
  
Besarnya nilai standar deviasi, yang melebihi nilai rata-rata data itu sendiri menunjukkan adanya ketimpangan yang ekstrem dipengaruhi oleh hadirnya beberapa data ekstrem (_outlier_) pada serangkaian data.

## D2. Potensi minimum dan maksimum pendapatan
  
Sebelum mulai mengambil keputusan dalam berjualan seblak secara _dropship_, saya perlu memerkirakan terlebih dahulu berapa potensi pendapatan yang sekiranya akan saya dapatkan dengan _dropship_ produk seblak dari Tokopedia.  

Hal ini dapat saya lakukan dengan membersihkan data outlier dari potensi pendapatan yang dapat saya terima dengan mengalikan harga produk dan total harga yang terjual.

In [6]:
lenTable = len(productsClean_df) # simpan ada berapa entri data di dalam tabel `productsClean_df`
listPotentialIncome = [] # buat list kosong penampung isian nilai yang akan menggambarkan potensi penjualan

# jalankan loop yang akan mengalikan entry harga dan angka banyaknya yang sudah terjual masing-masing index yang terepresentasikan melalui range length tabel
for index in range(lenTable):
    potentialIncome = productsClean_df['harga'][index] * productsClean_df['terjual'][index]
    listPotentialIncome.append(potentialIncome)

productsClean_df['potensi_pendapatan'] = listPotentialIncome # buat kolom baru berjudul 'potensi_interval'

# script untuk mengeluarkan output berupa nilai rentang atas dan rentang bawah potensi pendapatan dengan tingkat kepercayaan sebesar 95%
low, up = stats.norm.interval(confidence=0.95, loc=productsClean_df['potensi_pendapatan'].mean(), scale=stdevPrice/np.sqrt(lenTable))

print(f'Minimum potensi pendapatan dari hasil penjualan seblak dalam satu bulan (Rp): {low}')
print(f'Maximum potensi pendapatan dari hasil penjualan seblak dalam satu bulan (Rp): {up}')

Minimum potensi pendapatan dari hasil penjualan seblak dalam satu bulan (Rp): 6153331.762374804
Maximum potensi pendapatan dari hasil penjualan seblak dalam satu bulan (Rp): 6157210.7913317


Deskripsi:  
Berdasarkan hasil perhitungan nilai batas atas dan batas bawah penjualan dengan tingkat kepercayaan (confidence level) sebesar 95 persen, diketahui nilai minimum yang akan saya dapat ada di angka Rp6.153.331,62 per bulan. Sementara itu, diketahui nilai maksimum potensi hasil penjualan yang akan saya terima dengan berjualan/_dropship_ seblak di Tokopedia ialah sebesar Rp6.157.210,79

## D3. Perbedaan harga seblak yang dijual dari Jabodetabek dan luar Jabodetabek

Satu hal yang menarik untuk dieksplorasi ialah penjualan produk seblak di Tokopedia yang dapat dikategorikan secara lebih komprehensif, dengan membagi data ke dalam dua kategori area jual "Jabodetabek" dan "Luar Jabodetabek". Secara _common sense_, harga suatu produk jualan akan lebih mahal ketika barang tersebut dijual di tempat yang jauh dari area distribusi pusat. Sementara itu, kita mengenal asumsi bahwa Jakarta, terlebih area Jabodetabek, menjadi pusat atau sentra distribusi.  
  
Berdasarkan persoalan berikut, kita dapat melakukan uji statistik lebih lanjut dengan menerapkan uji statistik terhadap dua sampel independen. Kedua sampel tersebut berasal dari data yang masing-masing dikategorikan sebagai data harga produk yang dijual dari area Jabodetabek serta area luar Jabodetabek. Mengingat kedua kelompok sampel ini hakikatnya adalah dua kategori kolom 'area' yang berbeda, uji statistik yang tepat ialah dengan menggunakan uji Spearman.  
  
Hipotesis:  
**H0 :** Harga produk seblak yang dijual dari area Jabodetabek sama saja dengan yang dijual dari daerah luar Jabodetabek  
  
**H1 :** Ada hubungan antara harga produk seblak yang dijual di Tokopedia dengan lokasi asal penjualan produk (Jabodetabek dan luar Jabodetabek)  
  
Secara mudah, penulisan hipotesis dengan menggunakan notasi persamaan ialah sebagaimana berikut.  
  
> `H0: Harga Jabodetabek == Harga luar Jabodetabek`  
`H1: Harga Jabodetabek != Harga luar Jabodetabek`

In [7]:
# Cari nilai unik dalam kolom 'area'

productsClean_df['area'].unique()

array(['Luar Jabodetabek', 'Jabodetabek'], dtype=object)

In [9]:
priceMetro = productsClean_df['harga'][(productsClean_df['area'] == 'Jabodetabek')] # simpan tabel dengan memanggil kolom 'harga' dengan kriteria isi kolom 'area' adalah 'Jabodetabek'
priceNonMetro = productsClean_df['harga'][(productsClean_df['area'] == 'Luar Jabodetabek')] # simpan tabel dengan memanggil kolom 'harga' dengan kriteria isi kolom 'area' adalah 'Luar Jabodetabek'

In [10]:
tTestScore, pValueT = stats.ttest_ind(priceMetro, priceNonMetro) # lakukan uji two-pair t-test independent sample

print(f'Skor T-test: {tTestScore}, dengan angka p-Value: {pValueT}')

Skor T-test: 0.355545088171272, dengan angka p-Value: 0.7222950475907419


Kesimpulan:  
Dengan dilakukannya uji statistik dua sampel independen, diketahui p-value uji statistik (0.7222950475907419) bernilai lebih besar dari nilai kritis yang telah ditetapkan (0.05). Dari sini kita ketahui bahwa hipotesis 0 gagal untuk ditolak oleh uji statistik berikut (_fail to reject_). Hal ini mengartikan, harga produk seblak yang dijual oleh toko yang berasal dari area Jabodetabek maupun luar Jabodetabek via Tokopedia tidak terdapat adanya perbedaan yang signifikan.  

## D4. Hubungan antara harga dan popularitas produk  
  
Langkah selanjutnya adalah menentukan produk seblak yang akan bernilai ekonomis dengan memertimbangkan popularitasnya dengan tetap mengacu pada prinsip bisnis yang berupaya menekan modal seminimal mungkin. Hal ini dapat diraih melalui eksplorasi variabel/kolom harga dan bagaimana hubungannya terhadap nilai yang konsumen berikan terhadap produk tersebut.  
  
Cara ini dapat diraih dengan uji korelasi antar variabel. Umumnya, ada tiga jenis uji korelasi yang dapat dipilih sesuai dengan kondisi yang berlaku. Mengingat data pada kolom harga merupakan data kontinual dan nilai produk (rating) yang cenderung bersifat bukan kontinual, uji statistik korelasi yang tepat untuk soal berikut ialah uji korelasi Spearman's Rho.  
  
Berdasarkan kasus yang sedang diangkat, hipotesis yang dapat dibangun untuk melakukan uji berikut ialah sebagaimana berikut:  
  
`H0: Tidak ada hubungan yang signifikan antara harga dan popularitas dari sebuah produk seblak yang dijual di Tokopedia`  
`H1: Ada hubungan yang signifikan antara harga dan popularitas dari sebuah produk seblak yang dijual di Tokopedia`

In [13]:
# lakukan uji korelasi Spearman dengan memanggil fungsi berikut, lalu simpan hasilnya ke dalam dua variabel yang berbeda secara berurutan
corrSpearman, pValueS = stats.spearmanr(productsClean_df['harga'], productsClean_df['nilai_produk'])

print(f'Correlation: {corrSpearman}, at p-value: {pValueS}')

Correlation: 0.13646275147015283, at p-value: 0.0004343582123072226


Kesimpulan: Harga berperan dalam menentukan minat beli orang-orang terhadap seblak di Tokopedia. Hal ini dibuktikan dengan perolehan nilai p-value yang lebih kecil dari nilai _critical_ research yang telah ditetapkan sebelumnya (p-value: 0.0004343582123072226, critical value: 0.05). Dengan demikian, harga berkaitan dengan popularitas produk seblak.

# E. Kesimpulan

Berdasarkan serangkaian hasil observasi dan analisis data yang telah dilakukan, data etalase produk seblak yang dijual di platform Tokopedia cenderung bervariasi. Ada yang dijual mulai dari Rp350 hingga semahal lebih dari Rp200 ribu rupiah. Hal ini menjadikan kecenderungan distribusi harga produk yang tidak normal dan cenderung condong ke arah kiri. Artinya, hanya segelintir barang yang memang punya harga yang ekstrem dan di luar dari pola/harga pasaran umumnya. Sementara itu, temuan tes kurtosis juga mengindikasikan sebagian besar produk hanya berpusat pada satu titik harga tertentu saja, membuktikan bahwa harga seblak memiliki satu nominal ekuilibrium pasar serta dapat memiliki kasus harga yang spesial yang secara ekstrem lebih murah atau mahal, tetapi dengan nama-nama barang yang tidak banyak secara jumlah.

Hal ini mungkin saja terjadi mengingat produk seblak, selain umumnya dijual dalam bentuk kemasan, ada juga indikasi dijual secara grosir/kiloan. Perlu dicatat bahwa dengan melakukan kueri pencarian kata 'seblak' di platform Tokopedia, ada banyak kemungkinan hasil pencarian yang akan muncul, seperti halnya orang yang hanya menjual bahan baku seblak, bumbunya saja, maupun kemasannya pun dapat berpotensi masuk ke dalam hasil pengumpulan data dengan teknik _web scrapping_.  

Dengan menggunakan asumsi pendapatan bulanan atas berjualan seblak dengan mengalikan harga dan jumlah produk terjual, potensi pendapatan yang dapat diraih bervariasi. Oleh karenanya, pembersihan nilai outlier dari pengalian kedua variabel tersebut memberikan kesimpulan, setidaknya pendapatan penjualan seblak tiap bulan ada di antara Rp6.153.331.62 hingga Rp6.157,210,79. Hal ini sudah sesuai dengan tujuan utama yang ingin mendapatkan penghasilan tambahan berjualan seblak di angkat setidaknya Rp5.000.000 tiap bulannya selama 3 bulan mendatang. 

Dalam eksekusinya, _dropshipping_ tidak akan memertimbangkan lokasi toko sebagai basis penentuan produk yang akan dibeli. Hasil analisis menemukan, harga produk seblak tidak terpengaruh dengan lokasi penjual. Hal ini sangat krusial sehingga kendala seputar ketepatan waktu dan distribusi tidak akan mempengaruhi proses bisnis di masa mendatang.

Viralnya produk seblak juga akan dijadikan pertimbangan dalam melakukan langkah utama bisnis, dengan melihat aspek popularitas dari sebuah produk. Hasil temuan menunjukkan, pembeli cenderung menggemari produk seblak yang lebih murah. Uji korelasi Spearman's Rho menunjukkan, semakin murah produk seblak, semakin besar kecenderungannya untuk menjadi viral dan digemari. Hal ini akan sangat menunjang prospek bisnis yang sedang direncanakan. Ke depannya, bisnis seblak akan memuat produk-produk yang murah yang telah memiliki rating yang baik oleh pembeli-pembeli sebelumnya. 